<a href="https://colab.research.google.com/github/ncbx666/architecture-classification/blob/main/polina_darling_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install python-telegram-bot

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 702.3/702.3 kB 14.9 MB/s eta 0:00:00


In [4]:
!pip install nest_asyncio

In [ ]:
import os
import io
import asyncio
from PIL import Image
from fastai.vision.all import *
from telegram import Update
from telegram.ext import (
    Application,
    CommandHandler,
    MessageHandler,
    ContextTypes,
    filters
)

# 1. Загрузка модели
MODEL_PATH = '/content/architecture_style.pkl'
learn = load_learner(MODEL_PATH, cpu=True)

# 2. Обработка изображения (синхронная функция)
def process_image(image_bytes):
    img = Image.open(io.BytesIO(image_bytes)).convert('RGB')
    img_fastai = PILImage.create(io.BytesIO(image_bytes))
    pred_class, pred_idx, outputs = learn.predict(img_fastai)
    return str(pred_class)

# 3. Обработчики для Telegram бота
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text(
        "Привет! Отправь мне фото здания, и я определю его архитектурный стиль.\n"
        "Нажми /start чтобы увидеть это сообщение снова."
    )

async def handle_photo(update: Update, context: ContextTypes.DEFAULT_TYPE):
    photo = await update.message.photo[-1].get_file()
    photo_bytes = await photo.download_as_bytearray()

    try:
        # Запускаем синхронную функцию в отдельном потоке
        style = await asyncio.to_thread(process_image, photo_bytes)
        await update.message.reply_text(f"🔍 Это: {style}")
    except Exception as e:
        await update.message.reply_text(f"❌ Ошибка: {str(e)}")

# 4. Функция запуска бота
async def run_bot():
    TOKEN = "7339705790:AAEnBhiXNos2n-ydhRBa_hRKDmY1SvNmHFM"
    application = Application.builder().token(TOKEN).build()

    application.add_handler(CommandHandler("start", start))
    application.add_handler(MessageHandler(filters.PHOTO, handle_photo))

    print("Бот запущен...")
    await application.run_polling()

# 5. Запуск в Jupyter/Colab
try:
    import nest_asyncio
    nest_asyncio.apply()
    asyncio.run(run_bot())
except RuntimeError as e:
    if "event loop" in str(e):
        loop = asyncio.get_event_loop()
        loop.run_until_complete(run_bot())
    else:
        raise


Бот запущен...
